the goal of this notebook is to go back to what worked before (and did well on the learderboard) but fix batch normalization and input preprocessing. logically this should yield a higher score.

In [1]:
import os, glob, bcolz, gc

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

from keras import backend as K
from keras import optimizers
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, random_rotation, random_shear, random_zoom, random_shift, flip_axis

from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers.convolutional import MaxPooling2D, Convolution2D
from keras.layers.advanced_activations import PReLU, LeakyReLU

from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_v3 import preprocess_input as preprocess_input_incep_xcep
from keras.applications.imagenet_utils import preprocess_input as preprocess_input_vgg_resnet

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Using TensorFlow backend.


[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1057156453643440105, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 11332668621
 locality {
   bus_id: 2
 }
 incarnation: 11613837014834038207
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:85:00.0"]

In [2]:
def grab_optimizer(opt, lr):
    if opt == 'sgd':
        return optimizers.SGD(lr=lr, decay=1e-6, momentum=0.8, nesterov=True)
    elif opt == 'adam':
        return optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

def inception_block(filter_depth, inputs):
    t1 = Convolution2D(filter_depth, (1, 1), padding='same', activation=None,use_bias=False)(inputs)
    t1 = BatchNormalization()(t1)
    t1 = Activation('relu')(t1)
    
    tower_1 = Convolution2D(filter_depth, (3, 3), padding='same', activation=None, use_bias=False)(t1)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)
    
    tower_2 = Convolution2D(filter_depth, (5, 5), padding='same', activation=None, use_bias=False)(t1)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)
    
    tower_3 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    tower_3 = Convolution2D(filter_depth, (1, 1), padding='same', activation=None, use_bias=False)(tower_3)
    tower_3 = BatchNormalization()(tower_3)
    tower_3 = Activation('relu')(tower_3)
    
    return concatenate([tower_1, tower_2, tower_3], axis=3)

def conv_block(filter_depth, filter_size, pool_size, activation, drop_prob, inputs):
    x = Convolution2D(filter_depth, filter_size, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = MaxPooling2D(pool_size=pool_size)(x)
    x = Dropout(drop_prob)(x)
    return x

def dense_block(units, activation, drop_prob, inputs):
    x = Dense(units, activation=None)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Dropout(drop_prob)(x)
    return x

#my attempt at making a resnet identity block, wont be making any conv block
def resnet_block(filter_depth, filter_size, pool_size, activation, inputs):
    x = Convolution2D(filter_depth, (1,1), activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)

    x = Convolution2D(filter_depth, filter_size, activation=None, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)
    
    x = Convolution2D(filter_depth, (1,1), activation=None)(x)
    x = BatchNormalization()(x)
    
    shortcut = Convolution2D(filter_depth, (1,1))(inputs)
    shortcut = BatchNormalization()(shortcut)
    
    x = keras.layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def make_conv(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = conv_block(16, (3,3), (2,2),'relu', 0, inputs=inputs)
    m = conv_block(32, (3,3), (2,2), 'relu', 0, inputs=m)
    m = conv_block(64, (3,3), (2,2), 'relu', 0, inputs=m)
    m = conv_block(128, (3,3), (2,2), 'relu', 0, inputs=m)
    m = Flatten()(m)
    m = dense_block(1024, 'relu', 0.25, inputs=m)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def make_incep(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    
    # convolution preproccesing
    m = BatchNormalization()(inputs)
    m = Convolution2D(16, (3, 3), strides=(2,2), padding='valid', activation='relu')(m)
    m = MaxPooling2D((3, 3), strides=(2, 2))(m)
    #inception blocks
    m = BatchNormalization()(m)
    m = inception_block(32, m)
    m = inception_block(64, m)
    m = inception_block(128, m)
    m = inception_block(256, m)
    m = GlobalAveragePooling2D()(m)
    m = BatchNormalization()(m)
    m = dense_block(1024, 'relu', 0.25, inputs=m)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def make_resnet(input_shape, optimizer):
    inputs = Input(shape=input_shape)
    m = ZeroPadding2D((3,3))(inputs)
    m = Convolution2D(16, (3,3), strides=(2,2), activation='relu')(m)
    m = MaxPooling2D((3,3), strides=(2,2))(m)
    m = resnet_block(32, (3,3), (2,2), 'relu', inputs=m)
    m = resnet_block(64, (3,3), (2,2), 'relu', inputs=m)
    m = resnet_block(128, (3,3), (2,2), 'relu', inputs=m)
    m = AveragePooling2D((7, 7))(m)
    m = Flatten()(m)
    m = dense_block(1024, 'relu', 0.25, inputs=m)
    m = dense_block(1024, 'relu', 0.5, inputs=m)
    outputs = dense_block(1, 'sigmoid', 0, inputs=m)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# does not use precomputiation so it can use data augmentation
def train_kfolds(model, train_data, train_label, augment, model_out, model_init_weights, epochs, kfolds, batch_size):
    kf = KFold(n_splits=kfolds, shuffle=True)
    
    i = 0
    models_stats = {}
    for train_ixs, valid_ixs in kf.split(train_data):
        x_train = train_data[train_ixs]
        x_valid = train_data[valid_ixs]
        y_train = train_label[train_ixs]
        y_valid = train_label[valid_ixs]
        
        #augment the data
        if augment:
            for i in range(len(x_train)):
                x_train[i] = random_rotation(x_train[i], 20) 
                x_train[i] = random_shear(x_train[i], 0.2)
                x_train[i] = random_zoom(x_train[i], [0.8, 1.2])
                x_train[i] = random_shift(x_train[i], 0.2, 0.2)
        
        #re-initialzie the weights of the model on each run
        #by loading thi intiial stored weights from file
        model = load_model(model_init_weights)
        model_out_file = '{}_{}.model'.format(model_out, str(i))
        model_checkpoint = ModelCheckpoint(model_out_file, 
                                            monitor='val_loss', 
                                            save_best_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_acc',
                              patience=5,
                              verbose=1,
                              factor=0.1,
                              cooldown=10,
                              min_lr=0.00001)
        
        model.fit(x=x_train, y=y_train, 
                      batch_size=batch_size,
                      validation_data=(x_valid,y_valid),
                      epochs=epochs,
                      verbose=1,
                      callbacks=[model_checkpoint, reduce_lr])
        
        model = load_model(model_out_file)
        eval_tr = model.evaluate(x_train, y_train)
        eval_va = model.evaluate(x_valid, y_valid)
        tr_score = roc_auc_score(y_train, model.predict(x_train)[:, 0])
        va_score = roc_auc_score(y_valid, model.predict(x_valid)[:, 0])
        
        print('\n')
        print('kfold: {}'.format(str(i)))
        print('best model train acc: {}, loss: {}'.format(eval_tr[1], eval_tr[0]))
        print('best model valid acc: {}, loss: {}'.format(eval_va[1], eval_va[0]))
        print('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
        print('\n')
        models_stats[model_out_file] = {'score_tr_va':[tr_score, va_score], 'train_acc_loss':[eval_tr[1], eval_tr[0]], 'val_acc_loss':[eval_va[1], eval_va[0]]}
        
        with open(os.path.join(results_path,'{}_{}.out'.format(model_out,'history')), 'a') as f:
            f.write('kfold: {}'.format(str(i)))
            f.write('best model train acc: {}, loss: {}'.format(eval_tr[1], eval_tr[0]))
            f.write('best model valid acc: {}, loss: {}'.format(eval_va[1], eval_va[0]))
            f.write('best model train aroc score: {}, valid aroc score: {}'.format(tr_score, va_score))
            f.write('\n')
        
        i += 1
    
    return models_stats

In [3]:
path = os.path.join('/scratch', 'yns207', 'data_invasive')
test_path = os.path.join(path, 'test')
results_path = os.path.join(path, 'results')
train_path = os.path.join(path, 'train')
valid_path = os.path.join(path, 'valid')

In [4]:
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

train_img, test_img = read_imgs(300,400)

100%|██████████| 1531/1531 [00:49<00:00, 30.95it/s]


In [5]:
train_labels = np.array(train_set['invasive'].iloc[:])

In [6]:
%cd $path
batch_size = 64
epochs = 30
kfolds = 5
lr = 0.001

model_name = 'invasive_customconv_aug7'
init_weights_model = '{}_base.model'.format(model_name)

model = make_conv(train_img[0].shape, grab_optimizer('adam', lr))
model.save(init_weights_model)

proc_train_img = preprocess_input_vgg_resnet(train_img.astype(np.float32))

# train dense model on folds
performance = train_kfolds(model, proc_train_img, train_labels, False, model_name, init_weights_model, epochs, kfolds, batch_size)

/scratch/yns207/data_invasive
Train on 1836 samples, validate on 459 samples
Epoch 1/30
1836/1836 [==============================] - 31s - loss: 0.4913 - acc: 0.8050 - val_loss: 2.6105 - val_acc: 0.6405

ResourceExhaustedError: OOM when allocating tensor with shape[64,16,149,199]
	 [[Node: gradients_8/max_pooling2d_1_9/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@max_pooling2d_1_9/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/gpu:0"](activation_1_9/Relu, max_pooling2d_1_9/MaxPool, gradients_8/conv2d_2_9/convolution_grad/Conv2DBackpropInput)]]

Caused by op 'gradients_8/max_pooling2d_1_9/MaxPool_grad/MaxPoolGrad', defined at:
  File "/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-p